Louie Balderrama<br>
Springboard Data Science Career Track, January 2019 cohort<br>

<h1 align="center">JSON Based Data Exercise</h1>

1. Find the 10 countries with most projects

In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize

#setting up df
file = "world_bank_projects.json"
df = pd.read_json(file)

#grouping by country and aggregating by entry count
country_count = df.groupby("countryshortname").count()[[df.columns[0]]]
country_count.columns = ["count"]

#the top 10 countries with most projects
display(country_count.sort_values("count", ascending=False)[:10])

,count
countryshortname,
China,19
Indonesia,19
Vietnam,17
India,16
"Yemen, Republic of",13
Nepal,12
Bangladesh,12
Morocco,12
Mozambique,11


2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [2]:
#setting up df for nested json
data = json.load(open(file))
df = json_normalize(data, record_path="mjtheme_namecode")

#grouping by theme code and aggregating by entry count
code_count = df.groupby("code").count().reset_index()
code_count.sort_values("code", ascending=False)
code_count.columns = ["code", "count"]

#merging the dataframes to get the name column back
themes = pd.merge(df, code_count, on="code", how="left")
themes_drop = themes.drop_duplicates(subset="count", keep="first")
themes_drop = themes_drop.set_index("code")

#the top 10 major project themes
display(themes_drop.sort_values("count", ascending=False)[:10])

,name,count
code,,
11,,250
10,Rural development,216
8,Human development,210
2,Public sector governance,199
6,Social protection and risk management,168
4,Financial and private sector development,146
7,Social dev/gender/inclusion,130
5,Trade and integration,77
9,Urban development,50


3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## Solution A
Making use of a `lookup` dataframe and performing a *left join* to `themes` dataframe.

In [3]:
#creating lookup df
lookup = themes[themes.name != ""][["code", "name"]].drop_duplicates().set_index("code")

#merging lookup with themes df
themes_filled = pd.merge(themes[["code"]], lookup, on="code", how="left")

#merging themes_filled df with code_count df to replicate output of Item #2
themes_filled = pd.merge(code_count, themes_filled, on="code", how="left")
themes_filled = themes_filled.drop_duplicates(subset="count", keep="first")
themes_filled = themes_filled.set_index("code")[["name", "count"]]

#the top 10 major project themes, with corrected names
display(themes_filled.sort_values("count", ascending=False)[:10])

,name,count
code,,
11,Environment and natural resources management,250
10,Rural development,216
8,Human development,210
2,Public sector governance,199
6,Social protection and risk management,168
4,Financial and private sector development,146
7,Social dev/gender/inclusion,130
5,Trade and integration,77
9,Urban development,50


## Solution B
Updating the `themes` dataframe using *apply* method by referencing the `lookup` dataframe.

In [5]:
#using apply method to update themes df
themes["name"] = themes.apply(lambda row: \
                              lookup.loc[row[0]][0] if row["name"] == "" \
                              else row["name"], axis=1)

#merging themes df with code_count df to replicate output of Item #2
themes = pd.merge(code_count, themes, on=["code", "count"], how="left")
themes = themes.drop_duplicates(subset="count", keep="first")
themes = themes.set_index("code")[["name", "count"]]

#the top 10 major project themes, with corrected names
display(themes.sort_values("count", ascending=False)[:10])

,name,count
code,,
11,Environment and natural resources management,250
10,Rural development,216
8,Human development,210
2,Public sector governance,199
6,Social protection and risk management,168
4,Financial and private sector development,146
7,Social dev/gender/inclusion,130
5,Trade and integration,77
9,Urban development,50
